In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import (create_tool_calling_agent, 
                              AgentExecutor)

In [3]:
wikipedia_tool = WikipediaQueryRun(api_wrapper = WikipediaAPIWrapper())

In [4]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures", 
                     embedding_function = OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

retriever_tool = create_retriever_tool(retriever = retriever, 
                                       name = "Introduction-to-Data-and-Data-Science-Course-Lectures", 
                                       description = '''For any questions regarding 
                                       the Introduction to Data and Data Science course, you must use this tool.''')

/var/folders/fs/ly3y_3m57537dkrh09vsr_mw0000gn/T/ipykernel_3143/3325357122.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures",


In [5]:
@tool
def get_python_version() -> str:
    ''' Useful for questions regarding the version of Python currently used. '''
    return python_version()

In [6]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [7]:
chat = ChatOpenAI(model_name = 'gpt-4o-mini', 
                  temperature = 0,
                  seed = 365)

In [8]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
agent = create_tool_calling_agent(llm = chat,
                                  tools = tools,
                                  prompt = chat_prompt_template)

In [28]:
agent.invoke({"input":"Could you tell me the version of Python I'm currently using?", 
              "intermediate_steps":response["intermediate_steps"]})

AgentFinish(return_values={'output': 'You are currently using Python version 3.10.10.'}, log='You are currently using Python version 3.10.10.')

In [10]:
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               verbose=True, 
                               return_intermediate_steps=True)

In [11]:
response = agent_executor.invoke({"input":"Could you tell me the version of Python I'm currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.10You are currently using Python version 3.10.10.

> Finished chain.


In [24]:
response

{'input': "Could you tell me the version of Python I'm currently using?",
 'output': 'You are currently using Python version 3.10.10.',
 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Avaf4iD1wgZ06SBkzgT2PSl7', 'function': {'arguments': '{}', 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-fc45a2cf-5fda-4bc1-b87f-19c5c5ad12fc', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'call_Avaf4iD1wgZ06SBkzgT2PSl7', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': 'call_Avaf4iD1wgZ06SBkzgT2PSl7', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Avaf4iD1wgZ06SBkzgT2PSl7'),
   '3.10.10')]}